In [2]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Convolution1D, Flatten, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [3]:
# Carregando dados do IMDB Review Dataset
top_words = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [4]:
# Pré-processamento básico
max_review_length = 1600
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [5]:
import os
import tensorflow as tf
from keras.callbacks import TensorBoard

### Classe para salvar dados no Tensorboard

In [6]:
# Callback para dados do TensorBoard
class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./logs_v15', **kwargs):
        # Salvar dados de treino
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Salvar dados de validação
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_batch_end(self, epoch, logs=None):
        # Salvar dados a cada fim de batch
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()
        
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()

### Rede Neural: CNN

In [7]:
embedding_vector_length = 300
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(Convolution1D(64, 3, padding='same'))
model.add(Convolution1D(32, 3, padding='same'))
model.add(Convolution1D(16, 3, padding='same'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

In [8]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=3, callbacks=[TrainValTensorBoard(write_graph=False)], batch_size=64, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/3
20000/20000 [==============================] - 1396s 70ms/step - loss: 0.4194 - acc: 0.7950 - val_loss: 0.3155 - val_acc: 0.8676
Epoch 2/3
20000/20000 [==============================] - 1261s 63ms/step - loss: 0.1589 - acc: 0.9407 - val_loss: 0.3354 - val_acc: 0.8656
Epoch 3/3
20000/20000 [==============================] - 1442s 72ms/step - loss: 0.0473 - acc: 0.9853 - val_loss: 0.4202 - val_acc: 0.8544


In [9]:
# Avaliação na base de teste
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 84.54%
